Approach
The notebook demonstrates a comprehensive approach to implementing a Retrieval-Augmented Generation (RAG) model using the LangChain library with Hugging Face's language models. 

Frameworks and Libraries Used-
LangChain: A framework for building applications with large language models. It provides utilities for chaining together multiple language model calls and combining them with other data sources and APIs.
Hugging Face Transformers: A library that provides state-of-the-art general-purpose architectures for Natural Language Understanding (NLU) and Natural Language Generation (NLG).
Sentence Transformers: A library for easy-to-use sentence embeddings which can be used for tasks like clustering or semantic search.



In [1]:
pip install --upgrade --quiet pinecone-client pinecone-text pinecone-notebooks


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pinecone-text


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install langchain


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install langchain_community


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install langchain_huggingface


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
#replace your_api_key with your own api key 
api_key = "your_api_key"

In [7]:
from langchain_community.retrievers import PineconeHybridSearchRetriever


In [8]:
import os
from pinecone import Pinecone, ServerlessSpec
index_name = "hybrid-search-langchain-pinecone"

#initaializing pine cone client
pc = Pinecone(api_key=api_key)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [9]:
#creating index
if index_name not in pc.list_indexes().names():
   
    pc.create_index(

    name = index_name,
    dimension = 225,#dimesnion of dense vector
    metric = 'dotproduct',#sparse vetor supported for dotprodcut
    spec = ServerlessSpec(cloud = 'aws', region = 'us-east-1')
    )

UnauthorizedException: (401)
Reason: Unauthorized
HTTP response headers: HTTPHeaderDict({'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '85609a98a9b65560bf01070c06d67663', 'Date': 'Sat, 14 Sep 2024 07:44:19 GMT', 'Content-Type': 'text/html', 'Server': 'Google Frontend', 'Content-Length': '15', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: Invalid API Key


In [11]:
#pinecone index name
index = pc.Index(index_name)
index

Vector Embedding and Sparse matrix


In [12]:


#vector embedding
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name = 'all-MiniLM-L6-v2')
embeddings

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [13]:
#for sparse metrix
from pinecone_text.sparse import BM25Encoder
bm25_encoder = BM25Encoder()
bm25_encoder

In [14]:
conversation_history = [
    "1: User: Hi there! How are you doing today? | Bot: Hello! I'm doing great, thank you!",
    "2: User: What's the weather like today in New York? | Bot: Today in New York, it's sunny with a slight chance of rain.",
    "3: User: Great! Do you have any good lunch suggestions? | Bot: Sure! How about trying a new salad recipe?",
    "4: User: That sounds healthy. Any specific recipes? | Bot: You could try a quinoa salad with avocado and chicken.",
    "5: User: Sounds delicious! I'll try it. What about dinner? | Bot: For dinner, you could make grilled salmon with vegetables.",
    "6: User: Thanks for the suggestions! Any dessert ideas? | Bot: How about a simple fruit salad or yogurt with honey?",
    "7: User: Perfect! Now, what are some good exercises? | Bot: You can try a mix of cardio and strength training exercises.",
    "8: User: Any specific recommendations for cardio? | Bot: Running, cycling, and swimming are all excellent cardio exercises.",
    "9: User: I'll start with running. Can you recommend any books? | Bot: 'Atomic Habits' by James Clear is a highly recommended book.",
    "10: User: I'll check it out. What hobbies can I take up? | Bot: You could explore painting, hiking, or learning a new instrument.",
    "11: User: Hiking sounds fun! Any specific trails? | Bot: There are great trails in the Rockies and the Appalachian Mountains.",
    "12: User: I'll plan a trip. What about indoor activities? | Bot: Indoor activities like reading, cooking, or playing board games.",
    "13: User: Nice! Any good board games? | Bot: Settlers of Catan and Ticket to Ride are both excellent choices.",
    "14: User: I'll try them out. Any movie recommendations? | Bot: 'Inception' and 'The Matrix' are must-watch movies.",
    "15: User: I love those movies! Any TV shows? | Bot: 'Breaking Bad' and 'Stranger Things' are very popular.",
    "16: User: Great choices! What about podcasts? | Bot: 'How I Built This' and 'The Daily' are very informative.",
    "17: User: Thanks! What are some good travel destinations? | Bot: Paris, Tokyo, and Bali are amazing travel spots.",
    "18: User: I'll add them to my list. Any packing tips? | Bot: Roll your clothes to save space and use packing cubes.",
    "19: User: That's helpful! What about travel insurance? | Bot: Always get travel insurance for safety and peace of mind."
]


function to add embeddings to Pinecone

In [15]:
#TF-IDF values of the conversation_history
bm25_encoder.fit(conversation_history)

#store the values to a json file
bm25_encoder.dump('bm25_encoder_values.json')

#load to your BM25Encoder object
bm25_encoder = BM25Encoder().load('bm25_encoder_values.json')

100%|██████████| 19/19 [00:00<00:00, 2583.29it/s]


RAG MECHANISM

In [16]:
retriever = PineconeHybridSearchRetriever(embeddings= embeddings,sparse_encoder=bm25_encoder,index=index)

In [17]:
retriever

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x164db5670>, index=<pinecone.data.index.Index object at 0x1085f3e90>)

In [18]:
retriever.add_texts(conversation_history)

100%|██████████| 1/1 [00:04<00:00,  4.48s/it]


In [19]:
# 
retriever.invoke("I'll start with running.")

[Document(page_content="9: User: I'll start with running. Can you recommend any books? | Bot: 'Atomic Habits' by James Clear is a highly recommended book."),
 Document(page_content='8: User: Any specific recommendations for cardio? | Bot: Running, cycling, and swimming are all excellent cardio exercises.'),
 Document(page_content='7: User: Perfect! Now, what are some good exercises? | Bot: You can try a mix of cardio and strength training exercises.'),
 Document(page_content="10: User: I'll check it out. What hobbies can I take up? | Bot: You could explore painting, hiking, or learning a new instrument.")]

Retrieve Relevant Messages: A function to retrieve relevant messages from the conversation history based on a given prompt.



In [20]:
def retrieve_relevant_messages(test_prompt, history, index):
    # For the sake of this example, let's just return the last 3 messages
    return history[-3:]

def prepare_prompt(test_prompt, history, index_name='index', max_token_limit=225):
   
    # Retrieve relevant history messages
    relevant_history = retrieve_relevant_messages(test_prompt, conversation_history, index)
    
    # Combine the retrieved messages with the test prompt
    combined_prompt = "\n".join(relevant_history) + "\n" + test_prompt
    
    # Tokenize the combined prompt to ensure it does not exceed the maximum token limit
    tokens = combined_prompt.split()  # Simple tokenization by splitting on spaces
    if len(tokens) > max_token_limit:
        # If the combined prompt exceeds the token limit, truncate the history part
        excess_tokens = len(tokens) - max_token_limit
        history_tokens = "\n".join(relevant_history).split()
        truncated_history = " ".join(history_tokens[excess_tokens:])  # Truncate history
        combined_prompt = truncated_history + "\n" + test_prompt
    
    return combined_prompt, relevant_history


# Example usage
test_prompt = "Do you think it will help me stay fit?"
history = conversation_history



combined_prompt, context_referred = prepare_prompt(test_prompt, history)
print("Combined Prompt:\n", combined_prompt)
print("Context Referred:\n", context_referred)


Combined Prompt:
 17: User: Thanks! What are some good travel destinations? | Bot: Paris, Tokyo, and Bali are amazing travel spots.
18: User: I'll add them to my list. Any packing tips? | Bot: Roll your clothes to save space and use packing cubes.
19: User: That's helpful! What about travel insurance? | Bot: Always get travel insurance for safety and peace of mind.
Do you think it will help me stay fit?
Context Referred:
 ['17: User: Thanks! What are some good travel destinations? | Bot: Paris, Tokyo, and Bali are amazing travel spots.', "18: User: I'll add them to my list. Any packing tips? | Bot: Roll your clothes to save space and use packing cubes.", "19: User: That's helpful! What about travel insurance? | Bot: Always get travel insurance for safety and peace of mind."]
